In [1]:
# Creating a table in the Silver layer for storing the transformed order data
spark.sql("""
CREATE TABLE IF NOT EXISTS silver_orders (
    order_id STRING,
    customer_id STRING,
    product_id STRING,
    quantity INT,
    total_amount DOUBLE,
    transaction_date DATE,
    order_status STRING,
    last_updated TIMESTAMP
)
USING DELTA
""")


StatementMeta(, 7b218e4f-eb02-4260-bfb9-828b47360a3f, 3, Finished, Available, Finished)

DataFrame[]

In [3]:
# Get the timestamp of the last processed record to load only new or updated data
last_processed_df = spark.sql("SELECT MAX(last_updated) as last_processed FROM silver_orders")
last_processed_timestamp = last_processed_df.collect()[0]['last_processed']

if last_processed_timestamp is None:
    last_processed_timestamp = "1900-01-01T00:00:00.000+00:00"

StatementMeta(, 7b218e4f-eb02-4260-bfb9-828b47360a3f, 5, Finished, Available, Finished)

In [4]:
# Temporary view of the newly ingested order data from the Bronze layer by filtering records based on the last processed timestamp
spark.sql(f"""
CREATE OR REPLACE TEMPORARY VIEW bronze_incremental_orders AS
SELECT *
FROM bronzelayer.orders WHERE ingestion_timestamp > '{last_processed_timestamp}'
""")

StatementMeta(, 7b218e4f-eb02-4260-bfb9-828b47360a3f, 6, Finished, Available, Finished)

DataFrame[]

In [5]:
# Apply the required transformations to clean and standardize the data
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW silver_incremental_orders AS
SELECT
    transaction_id as order_id,
    customer_id,
    product_id,
    CASE
        WHEN quantity < 0 THEN 0
        ELSE quantity
    END AS quantity,
    CASE
        WHEN total_amount < 0 THEN 0
        ELSE total_amount
    END AS total_amount,
    CAST(transaction_date AS DATE) AS transaction_date,
    CASE
        WHEN quantity = 0 AND total_amount = 0 THEN 'Cancelled'
        WHEN quantity > 0 AND total_amount > 0 THEN 'Completed'
        ELSE 'In Progress'
    END AS order_status,
    CURRENT_TIMESTAMP() AS last_updated
FROM bronze_incremental_orders
WHERE transaction_date IS NOT NULL 
  AND customer_id IS NOT NULL 
  AND product_id IS NOT NULL
""")

StatementMeta(, 7b218e4f-eb02-4260-bfb9-828b47360a3f, 7, Finished, Available, Finished)

DataFrame[]

In [6]:
# MERGE statement to upsert the cleaned and transformed data into the Silver layer, ensuring both new and updated records are handled properly
spark.sql("""
MERGE INTO silver_orders target
USING silver_incremental_orders source
ON target.order_id = source.order_id
WHEN MATCHED THEN
    UPDATE SET *
WHEN NOT MATCHED THEN
    INSERT *
""")


StatementMeta(, 7b218e4f-eb02-4260-bfb9-828b47360a3f, 8, Finished, Available, Finished)

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [8]:
# Verify the data in the Silver layer
spark.sql("SELECT * FROM silver_orders LIMIT 10").show()


StatementMeta(, 7b218e4f-eb02-4260-bfb9-828b47360a3f, 10, Finished, Available, Finished)

+---------+-----------+----------+--------+------------+----------------+------------+--------------------+
| order_id|customer_id|product_id|quantity|total_amount|transaction_date|order_status|        last_updated|
+---------+-----------+----------+--------+------------+----------------+------------+--------------------+
|TRX001128|        240|       422|       7|         0.0|      2020-07-24| In Progress|2025-01-23 09:20:...|
|TRX001179|         85|       434|       7|         0.0|      2023-05-28| In Progress|2025-01-23 09:20:...|
|TRX001600|        879|       134|       7|         0.0|      2022-07-02| In Progress|2025-01-23 09:20:...|
|TRX003550|        917|       889|       7|         0.0|      2023-04-10| In Progress|2025-01-23 09:20:...|
|TRX005252|        942|       402|       7|         0.0|      2021-12-10| In Progress|2025-01-23 09:20:...|
|TRX009297|        475|       869|       7|         0.0|      2022-03-24| In Progress|2025-01-23 09:20:...|
|TRX009266|        983|     